In [ ]:
import json
import math
import re

import pandas as pd
import numpy as np
import folium

In [ ]:
creativeeurope = pd.read_excel('data/creative/input/CreativeEurope_Projects_Overview_2018-08-01.xls')
creativeeurope.head()

In [ ]:
list(creativeeurope)

In [ ]:
creativeeurope.columns = [
    re.sub(r'^Partner (\d+) (.+)$', r'Partner_\2_\1', column)
    for column in creativeeurope.columns
]
creativeeurope.head()

In [ ]:
[creativeeurope.shape, creativeeurope['Project Number'].nunique()]

In [ ]:
creativeeurope_long = pd.wide_to_long(
    creativeeurope,
    ['Partner_name','Partner_organisation type', 'Partner_address', 'Partner_country', 'Partner_region', 'Partner_website'],
    'Project Number', 'partner_number',
    sep='_'
)
creativeeurope_long

In [ ]:

creativeeurope_long_uk = creativeeurope_long[creativeeurope_long.Partner_country == 'UK']
creativeeurope_long_uk.shape

In [ ]:
creativeeurope_long_uk['postcode'] = creativeeurope_long_uk.Partner_address.str.extract(r'.*\b(\w{1,4}\s*\d\w\w)\b.*')[0].str.strip().str.replace(' ', '')

creativeeurope_long_uk.head()

In [ ]:
postcodes = pd.read_csv('data/postcodes/input/ukpostcodes.csv')

In [ ]:
postcodes['postcode'] = postcodes.postcode.str.strip().str.replace(' ', '')

In [ ]:
creativeeurope_long_uk =  pd.merge(creativeeurope_long_uk, postcodes,  left_on=['postcode'], right_on=['postcode'])

creativeeurope_long_uk.head()

In [ ]:
list(creativeeurope_long_uk)

In [ ]:
def make_creative_data_geo_json(coordis_data):
    def make_feature(row):
        properties = {
            property: row[property]
            for property in ['Project Title', 'Partner_name', 'Partner_website', "EU Grant award in euros (This amount represents the grant awarded after the selection stage and is indicative. Please note that any changes made during or after the project's lifetime will not be reflected here.)", "Coordinator's name"]
            if str(row[property]) != 'nan'
        }

        return {
            'type': 'Feature',
            'geometry': {
                "type": "Point",
                "coordinates": [row['longitude'], row['latitude']]
            },
            'properties': properties
        }
    features = list(coordis_data.apply(make_feature, axis=1))
    return { 'type': 'FeatureCollection', 'features': features }
with open('data/creative/output/creative_data.geo.json', 'w') as file:
    json.dump(make_creative_data_geo_json(
        creativeeurope_long_uk[~pd.isnull(creativeeurope_long_uk['latitude'])]
    ), file, sort_keys=True)

In [ ]:
creativeeurope.shape

In [ ]:
creativeeurope = creativeeurope.rename(index=str, columns={"Coordinator's address": "coordinators_address", "Coordinator's country": "coordinators_country"})
creativeeurope = creativeeurope[creativeeurope.coordinators_country == 'UK']

creativeeurope['postcode'] = creativeeurope.coordinators_address.str.extract(r'.*\b(\w{1,4}\s*\d\w\w)\b.*')[0].str.strip().str.replace(' ', '')

creativeeurope.head()

In [ ]:
creativeeurope =  pd.merge(creativeeurope, postcodes,  left_on=['postcode'], right_on=['postcode'])
 
creativeeurope.head()